In [1]:
from typing import Any

import lightning.pytorch as pl
import torch
from lightning.pytorch.callbacks import ModelCheckpoint
import mlflow
from torch.utils.data import DataLoader, Dataset

from src.ml.data.splitting import create_data_splits
from src.ml.modeling import (
    model_factory,
    optimizer_factory,
)
from src.ml.utils.set_seed import set_seed

from datetime import datetime
from loguru import logger
from pathlib import Path
from shutil import copy, copytree, rmtree

import yaml

from src.ml.train_neural_network import train_neural_network
from src.ml.data import data_sets_factory
from src.ml.preprocessing import preprocessing_factory
from src.ml.utils.set_seed import set_seed

In [2]:
CONFIG_FILE = Path("src/ml/config.yaml")

In [3]:
torch.set_default_device(torch.device("cpu"))

In [4]:
# load config file

logger.info("Loading config file.")

with open(CONFIG_FILE, "r") as f:
    config = yaml.safe_load(f)

2024-12-13 02:53:42.388 | INFO     | __main__:<module>:3 - Loading config file.


In [5]:
# load data

logger.info("Loading data.")

data_sets = data_sets_factory(**config["data_set"])
logger.info("Loaded {} data sets.", len(data_sets))

# preprocess data

logger.info("Start preprocessing.")

for preprocessing_step in config["preprocessing"]:
    logger.info("Perform {} preprocessing.", preprocessing_step["name"])

    transform = preprocessing_factory(**preprocessing_step)
    data_sets = [transform(data_set) for data_set in data_sets]

dataset = data_sets[0]

2024-12-13 02:53:42.393 | INFO     | __main__:<module>:3 - Loading data.
2024-12-13 02:53:42.394 | INFO     | __main__:<module>:6 - Loaded 1 data sets.
2024-12-13 02:53:42.394 | INFO     | __main__:<module>:10 - Start preprocessing.


In [6]:
train_dataset, val_dataset, test_dataset = create_data_splits(
    dataset, **config["training"]["splitting_config"]
)

In [7]:
train_loader = DataLoader(train_dataset, **config["training"]["dataloader_config"])
test_loader = DataLoader(test_dataset,batch_size=1)
val_loader = DataLoader(val_dataset)

optimizer = optimizer_factory(**config["training"]["optimizer_config"])
model = model_factory(
    optimizer=optimizer,
    dim=len(train_dataset[0]["branch_lengths"]),
    **config["training"]["model_config"],
)

In [8]:
from src.ml.modeling.conditional_tree_flow import ConditionalTreeFlow

model = ConditionalTreeFlow.load_from_checkpoint("ml_data/models/debug_simple_conditional_flow/epoch=98-val_loss=-10161.30.ckpt")

In [9]:
sample = next(iter(test_loader))
sample

{'branch_lengths': tensor([[-1.8104e-03, -6.8320e-04,  4.2604e-05, -5.3133e-04,  3.2836e-05,
           2.1246e-04, -1.0737e-03, -2.1067e-04, -6.0958e-04,  1.3781e-03]]),
 'clades_one_hot': tensor([[1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
          1., 0.]])}

In [10]:
latent = model.forward(sample)
latent

{'z': tensor([[-1.0879,  0.0057, -0.5851, -0.6469, -0.4700,  1.0595, -1.3038,  0.5304,
           1.4702,  1.1122]], grad_fn=<MulBackward0>),
 'context': tensor([[1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
          1., 0.]]),
 'log_dj': tensor([53.9412], grad_fn=<AddBackward0>)}

In [11]:
output = model.inverse(latent)
output

{'branch_lengths': tensor([[-1.8105e-03, -6.8316e-04,  4.2498e-05, -5.3146e-04,  3.2805e-05,
           2.1242e-04, -1.0737e-03, -2.1067e-04, -6.0964e-04,  1.3781e-03]],
        grad_fn=<SubBackward0>),
 'clades_one_hot': tensor([[1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
          1., 0.]]),
 'log_dj': tensor([53.9412], grad_fn=<AddBackward0>)}

In [27]:
prior = model.prior.sample((10,))
latent["z"] = prior.clone()
output = model.inverse(latent)
output

{'branch_lengths': tensor([-0.0018, -0.1930,  0.5294, -0.5510,  0.0073, -0.2338, -0.0323,  0.0152,
          0.0274, -0.0170], grad_fn=<SubBackward0>),
 'clades_one_hot': tensor([[1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
          1., 0.]]),
 'log_dj': tensor([53.9412], grad_fn=<AddBackward0>)}